<a href="https://colab.research.google.com/github/Saisrujan4/NNDL/blob/main/Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [55]:
data=pd.read_csv('/content/drive/MyDrive/LD.csv')
data.head()

,Date,Time,Load (kW),Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,01.09.2018,0:00,5551.82208,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1:00,4983.17184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2:00,4888.39680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3:00,5072.95872,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,4:00,5196.25980,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
X = []
Y = []
for i in range(24, data.shape[0]):
    X.append(data['Load (kW)'].iloc[i-25]) 
    Y.append(data['Load (kW)'].iloc[i])


In [57]:
data = pd.DataFrame({'Previous Day Load' : X, 'Present Day Load' : Y})
data.head()


,Previous Day Load,Present Day Load
0,6938.37468,4931.26380
1,5551.82208,4775.53968
2,4983.17184,4713.81120
3,4888.39680,4689.02628
4,5072.95872,4844.75040


In [58]:

Xmax = np.max(data['Previous Day Load'])
Xmin = np.min(data['Previous Day Load'])
Ymax = np.max(data['Present Day Load'])

Ymin = np.min(data['Present Day Load'])
data['Previous Day Load'] = (data['Previous Day Load'] - Xmin) / (Xmax - Xmin)
data['Present Day Load'] = (data['Present Day Load'] - Ymin) / (Ymax - Ymin)
data.head()

,Previous Day Load,Present Day Load
0,0.651650,0.284300
1,0.397877,0.255799
2,0.293800,0.244501
3,0.276454,0.239965
4,0.310234,0.268466


In [59]:

x = data['Previous Day Load']
y = data['Present Day Load']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x, y, test_size=0.1, random_state=101)


In [60]:
m = np.random.uniform(-2, 2)
c = np.random.uniform(-2, 2)
eta = 0.1
epochs = 500
gm = 0.9
Em2 = 0
Ec2 = 0
e = 1E-6
for _ in range(epochs):
    for i in range(Xtrain.shape[0]):
        gradM = -1 * (Ytrain.iloc[i] - m * Xtrain.iloc[i] - c) * Xtrain.iloc[i]
        gradC = -1 * (Ytrain.iloc[i] - m * Xtrain.iloc[i] - c)
        
        Em2 = (gm*Em2) + ((1-gm)*(gradM ** 2))
        Ec2 = (gm*Ec2) + ((1-gm)*(gradC ** 2))
        
        m -= (eta * gradM) / ((e + Em2) ** 0.5)
        c -= (eta * gradC) / ((e + Ec2) ** 0.5)
print("m = ",m)

print("c = ",c)


m =  0.7459993025267385
c =  0.14754753492225675


In [61]:

train_pred = [m * Xtrain.iloc[i] + c for i in range(Xtrain.shape[0])]
test_pred = [m * Xtest.iloc[i] + c for i in range(Xtest.shape[0])]


In [62]:


train_pred = [i * (Ymax - Ymin) + Ymin for i in train_pred]
test_pred = [i * (Ymax - Ymin) + Ymin for i in test_pred]
trainY = [i * (Ymax - Ymin) + Ymin for i in Ytrain]
testY = [i * (Ymax - Ymin) + Ymin for i in Ytest]

In [63]:



train_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
train_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
train_RMSE = train_MSE ** 0.5
print('Training error :')
print(f'MAE : {train_MAE}')
print(f'MSE : {train_MSE}')

print(f'RMSE : {train_RMSE}')


Training error :
MAE : 631.4166200569462
MSE : 691375.3194148488
RMSE : 831.4898191889356


In [64]:

test_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
test_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
test_RMSE = test_MSE ** 0.5
print('Testing error:')
print(f'MAE : {test_MAE}')
print(f'MSE :{test_MSE}')
print(f'RMSE : {test_RMSE}')


Testing error:
MAE : 613.0434535946625
MSE :662035.6945268413
RMSE : 813.6557592291972


In [65]:
x = float(input('Enter load at an hour on previous day: '))
x = (x - Xmin) / (Xmax - Xmin)
pred = m * x + c
pred = (pred * (Ymax - Ymin)) + Ymin
print('predicted load at present hour = ', pred)

Enter load at an hour on previous day: 2000
predicted load at present hour =  3156.1553656878536
